In [83]:
import sqlite3
from collections import defaultdict
con = sqlite3.connect('mxm_dataset.db')

cur = con.cursor()


with open("mxm_779k_matches.txt", "r") as f:
    idMatcherList = f.readlines()
    
while idMatcherList[0][0] == "#":
    idMatcherList.pop(0)
    
idMatcher = dict()
    
for i in range(len(idMatcherList)):
    songList = idMatcherList[i].split("<SEP>")
    idMatcher[songList[0]] = {"artist":songList[1], "songName":songList[2]}

def findSong(id):
    return idMatcher[id]

In [85]:

print(cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall())
# print(cur.fetchall())

print(cur.execute('PRAGMA table_info(lyrics);').fetchall())

# for row in cur.execute('SELECT * FROM lyrics limit 1000'):
#         print(row)
def tfidfQuery(sentence):
    songImportances = defaultdict(int)
    words = sentence.split(" ")
    for word in words:
        songImportance = tfidf(word)
        for key in songImportance:
#             print("wow", key)
#             print(songImportance[key])
            songImportances[key] += songImportance[key]
#     print(songImportances)
    bestMatch = max(songImportances.items(), key=lambda x:x[1])
    return [bestMatch[0], findSong(bestMatch[0]), songImportance[bestMatch[0]]]
        
def tfidf(word):
    songImportance = defaultdict(int)
    results = cur.execute("SELECT track_id, count from lyrics where word='" + word + "';").fetchall()
    sumRes = cur.execute("SELECT count(count) from lyrics where word='" + word + "';").fetchall()[0][0]
    nDocs = cur.execute("SELECT count(count) from lyrics").fetchall()[0][0]
    print(nDocs)
    print(sumRes)
    for song in results:
        songImportance[song[0]] += song[1]*nDocs/sumRes
        
    bestMatch = max(songImportance, key=lambda x:x[1]), 
    return songImportance
# [bestMatch[0], songImportance[bestMatch[0]]]



(tfidfQuery("love"))
    

    


[('words',), ('lyrics',)]
[(0, 'track_id', '', 0, None, 0), (1, 'mxm_tid', 'INT', 0, None, 0), (2, 'word', 'TEXT', 0, None, 0), (3, 'count', 'INT', 0, None, 0), (4, 'is_test', 'INT', 0, None, 0)]
19045332
73423


['TRJVUJV128F4266767',
 {'artist': 'Jessica Simpson', 'songName': "I Think I'm In Love With You"},
 39168.177982376095]